In [2]:
import pandas as pd
import quandl
import math
import numpy as np
from sklearn import preprocessing,cross_validation,svm
from sklearn.linear_model import LinearRegression

C:\Users\Rishabh\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
df = quandl.get('NSE/INFY')
print(df)

               Open     High      Low     Last    Close  Total Trade Quantity  \
Date                                                                            
2011-06-29  2868.00  2907.00  2868.00  2880.00  2881.75              600094.0   
2011-06-30  2886.05  2919.00  2881.50  2903.10  2910.45             1417730.0   
2011-07-01  2935.00  2969.95  2924.10  2928.00  2934.15              800308.0   
2011-07-04  2966.65  2977.00  2935.00  2941.00  2938.95              514324.0   
2011-07-05  2957.00  2969.90  2926.00  2959.00  2956.45              802966.0   
2011-07-06  2947.00  2972.45  2937.10  2954.50  2953.70              430028.0   
2011-07-07  2955.20  3000.00  2943.00  2998.00  2995.70              718934.0   
2011-07-08  3000.00  3023.90  2967.25  2967.25  2976.55              708724.0   
2011-07-11  2953.90  2976.10  2888.55  2926.35  2921.15             1150815.0   
2011-07-12  2850.00  2850.00  2746.80  2786.10  2791.55             3706582.0   
2011-07-13  2766.00  2807.35

In [4]:
df = df[['Open','High','Close','Total Trade Quantity']]
print(df.head())

               Open     High    Close  Total Trade Quantity
Date                                                       
2011-06-29  2868.00  2907.00  2881.75              600094.0
2011-06-30  2886.05  2919.00  2910.45             1417730.0
2011-07-01  2935.00  2969.95  2934.15              800308.0
2011-07-04  2966.65  2977.00  2938.95              514324.0
2011-07-05  2957.00  2969.90  2956.45              802966.0


In [5]:
df['HC_PCT']=(df['High']-df['Close'])/df['Open']*100.0
print(df.head())

               Open     High    Close  Total Trade Quantity    HC_PCT
Date                                                                 
2011-06-29  2868.00  2907.00  2881.75              600094.0  0.880404
2011-06-30  2886.05  2919.00  2910.45             1417730.0  0.296253
2011-07-01  2935.00  2969.95  2934.15              800308.0  1.219761
2011-07-04  2966.65  2977.00  2938.95              514324.0  1.282591
2011-07-05  2957.00  2969.90  2956.45              802966.0  0.454853


In [6]:
df['PCT_change'] = (df['Close']-df['Open'])/df['Open']*100.0
print(df.head())

               Open     High    Close  Total Trade Quantity    HC_PCT  \
Date                                                                    
2011-06-29  2868.00  2907.00  2881.75              600094.0  0.880404   
2011-06-30  2886.05  2919.00  2910.45             1417730.0  0.296253   
2011-07-01  2935.00  2969.95  2934.15              800308.0  1.219761   
2011-07-04  2966.65  2977.00  2938.95              514324.0  1.282591   
2011-07-05  2957.00  2969.90  2956.45              802966.0  0.454853   

            PCT_change  
Date                    
2011-06-29    0.479428  
2011-06-30    0.845446  
2011-07-01   -0.028961  
2011-07-04   -0.933713  
2011-07-05   -0.018600  


In [19]:
df = df[['Close','HC_PCT','PCT_change','Total Trade Quantity']]
print(df.head(18))
print(len(df))

              Close    HC_PCT  PCT_change  Total Trade Quantity
Date                                                           
2011-06-29  2881.75  0.880404    0.479428              600094.0
2011-06-30  2910.45  0.296253    0.845446             1417730.0
2011-07-01  2934.15  1.219761   -0.028961              800308.0
2011-07-04  2938.95  1.282591   -0.933713              514324.0
2011-07-05  2956.45  0.454853   -0.018600              802966.0
2011-07-06  2953.70  0.636240    0.227350              430028.0
2011-07-07  2995.70  0.145506    1.370466              718934.0
2011-07-08  2976.55  1.578333   -0.781667              708724.0
2011-07-11  2921.15  1.860253   -1.108704             1150815.0
2011-07-12  2791.55  2.050877   -2.050877             3706582.0
2011-07-13  2777.30  1.086406    0.408532             1667271.0
2011-07-14  2740.35  0.856806   -0.720949             1361403.0
2011-07-15  2731.35  1.151603   -0.461006              799078.0
2011-07-18  2713.40  1.091408   -0.78976

In [22]:
forecast_col='Close'
df.fillna(-99999, inplace=True)

forecast_out = int(math.ceil(0.01*len(df)))
print("forecast: ",forecast_out)
df['label']=df[forecast_col].shift(-forecast_out)
df.dropna(inplace=True)
print(df)

forecast:  17
              Close    HC_PCT  PCT_change  Total Trade Quantity    label
Date                                                                    
2011-06-29  2881.75  0.880404    0.479428              600094.0  2828.25
2011-06-30  2910.45  0.296253    0.845446             1417730.0  2807.75
2011-07-01  2934.15  1.219761   -0.028961              800308.0  2801.80
2011-07-04  2938.95  1.282591   -0.933713              514324.0  2796.65
2011-07-05  2956.45  0.454853   -0.018600              802966.0  2751.10
2011-07-06  2953.70  0.636240    0.227350              430028.0  2775.90
2011-07-07  2995.70  0.145506    1.370466              718934.0  2815.10
2011-07-08  2976.55  1.578333   -0.781667              708724.0  2750.95
2011-07-11  2921.15  1.860253   -1.108704             1150815.0  2732.60
2011-07-12  2791.55  2.050877   -2.050877             3706582.0  2709.15
2011-07-13  2777.30  1.086406    0.408532             1667271.0  2591.20
2011-07-14  2740.35  0.856806   -0.72

In [28]:
x = np.array(df.drop(['label'],1))
y = np.array(df['label'])
print(len(x),len(y))
print(x)
print(np.mean(x))
x = preprocessing.scale(x)
print("________________")
print(x)
print("________________")
print(len(x),len(y))

1666 1666
[[  2.88175000e+03   8.80404463e-01   4.79428173e-01   6.00094000e+05]
 [  2.91045000e+03   2.96252664e-01   8.45446198e-01   1.41773000e+06]
 [  2.93415000e+03   1.21976150e+00  -2.89608177e-02   8.00308000e+05]
 ..., 
 [  1.18250000e+03   5.00000000e-01   2.11864407e-01   3.05075000e+06]
 [  1.17190000e+03   8.88324873e-01  -8.54483926e-01   8.97741300e+06]
 [  1.14675000e+03   2.89948454e+00  -1.48195876e+00   3.76331000e+06]]
628723.166278
________________
[[ 0.84650219 -0.17845327  0.34606335 -0.55781835]
 [ 0.87660177 -0.80075817  0.60636921 -0.31936649]
 [ 0.90145753  0.18306841 -0.01549435 -0.4994288 ]
 ..., 
 [-0.93561324 -0.58370335  0.1557765   0.15688047]
 [-0.94673016 -0.17001555 -0.60259249  1.8853071 ]
 [-0.97310662  1.97250032 -1.04884204  0.36468842]]
___________
1666 1666


In [31]:
x_train,x_test,y_train,y_test = cross_validation.train_test_split(x,y,test_size=0.2)
clf = LinearRegression()
clf.fit(x_train,y_train)
accuracy = clf.score(x_test,y_test)
print(accuracy)

0.942658172738


In [32]:
x_train,x_test,y_train,y_test = cross_validation.train_test_split(x,y,test_size=0.2)
clf = svm.SVR()
clf.fit(x_train,y_train)
accuracy = clf.score(x_test,y_test)
print(accuracy)

0.316241443654


In [2]:
import keras

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "C:\Users\Rishabh\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 14, in swig_import_helper
    return importlib.import_module(mname)
  File "C:\Users\Rishabh\AppData\Local\Enthought\Canopy\edm\envs\User\lib\importlib\__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 986, in _gcd_import
  File "<frozen importlib._bootstrap>", line 969, in _find_and_load
  File "<frozen importlib._bootstrap>", line 958, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 666, in _load_unlocked
  File "<frozen importlib._bootstrap>", line 577, in module_from_spec
  File "<frozen importlib._bootstrap_external>", line 906, in create_module
  File "<frozen importlib._bootstrap>", line 222, in _call_with_frames_removed
ImportError: DLL load failed with error code -107

In [ ]:
impoer